# Install packages for code to run in colab

In [1]:
!pip install -U "scipy>=1.10"
!pip install git+https://github.com/sumeetkhatri/QuTIpy
!pip -q install cvxpy scs

  Cloning https://github.com/sumeetkhatri/QuTIpy to /tmp/pip-req-build-ryk4g78c
  Running command git clone --filter=blob:none --quiet https://github.com/sumeetkhatri/QuTIpy /tmp/pip-req-build-ryk4g78c
  Resolved https://github.com/sumeetkhatri/QuTIpy to commit fd16eb8bfde03de385af5385e8a2582ec13794c8
  Preparing metadata (setup.py) ... done
  Created wheel for QuTIpy: filename=QuTIpy-0.1.0a0-py3-none-any.whl size=65700 sha256=e02eaafeb08707251eccdca3c8a281e651f3da40fb9358a3f9e12cb8c573984b
  Stored in directory: /tmp/pip-ephem-wheel-cache-6bh__dy1/wheels/b9/02/f6/62f07f18161277b23b70a6075a95dd3b1cf20c7ed51c3bcbfb
Successfully built QuTIpy


# k_positivity.py

(optimization adjusted to work with SCS instead of MOSEK because I don't have a MOSEK license)

In [2]:
# If needed, install deps first:
# !pip -q install cvxpy scs scipy qutipy

import numpy as np
import itertools
import cvxpy as cvx
from scipy.linalg import expm
from qutipy import *

# ---- solver helper: avoids requiring MOSEK ----
def solve_alt(prob, **kwargs):
    """
    Solve a CVXPY problem without needing MOSEK.
    Prefers SCS (open-source), falls back if unavailable
    """
    installed = set(cvx.installed_solvers())
    if "SCS" in installed:
        opts = dict(kwargs)
        opts.setdefault("eps", 1e-6)
        opts.setdefault("max_iters", 50_000)
        return prob.solve(solver=cvx.SCS, **opts)
    elif "CLARABEL" in installed:
        return prob.solve(solver=cvx.CLARABEL, **kwargs)
    elif "ECOS" in installed:
        return prob.solve(solver=cvx.ECOS, **kwargs)
    else:
        return prob.solve(**kwargs)


def bilinear_opt_relax(H, dA, dB, PPT=False):
    '''
    Optimization of the form
        min_H Tr(H(ρ ⊗ σ)),
    with respect to states ρ and σ. We
    relax this to operators P that satisfy
    the appropriate partial trace and
    partial transpose conditions.
    '''
    P = cvx.Variable((dA*dB, dA*dB), hermitian=True)
    rho = cvx.Variable((dA, dA), hermitian=True)
    sigma = cvx.Variable((dB, dB), hermitian=True)

    c = [P >> 0, rho >> 0, sigma >> 0]
    c += [cvx.trace(rho) == 1, cvx.trace(sigma) == 1]
    c += [cvx.partial_trace(P, [dA, dB], 0) == sigma,
          cvx.partial_trace(P, [dA, dB], 1) == rho]
    if PPT:
        c += [cvx.partial_transpose(P, [dA, dB], 0) >> 0]

    obj = cvx.Minimize(cvx.real(cvx.trace(H @ P)))
    prob = cvx.Problem(obj, constraints=c)

    solve_alt(prob)  # was: prob.solve(eps=1e-9, solver=cvx.MOSEK)
    return prob.value


def bilinear_opt_relax_hierarchy(H, dA, dB, k=1):
    M = {}
    S = {}

    rho = cvx.Variable((dA, dA), hermitian=True)
    sigma = cvx.Variable((dB, dB), hermitian=True)

    c = [rho >> 0, cvx.trace(rho) == 1]
    c += [sigma >> 0, cvx.trace(sigma) == 1]

    for j in range(1, k+1):
        M[j] = cvx.Variable((dA * dB**j, dA * dB**j), hermitian=True)
        S[j] = list(itertools.permutations(range(1, j+1)))
        c += [M[j] >> 0]
        c += [M[j] == cvx.kron(eye(dA), permute_tensor_factors(pi, [dB]*j)) @ M[j] @
              cvx.kron(eye(dA), dag(permute_tensor_factors(pi, [dB]*j)))
              for pi in S[j]]

    c += [cvx.partial_trace(M[1], [dA, dB], 0) == rho,
          cvx.partial_trace(M[1], [dA, dB], 1) == sigma]

    for j in range(2, k+1):
        c += [cvx.partial_trace(M[j], [dA * dB**(j-1), dB], 1) == M[j-1]]

    obj = cvx.Minimize(cvx.real(cvx.trace(H @ M[1])))
    prob = cvx.Problem(obj, constraints=c)

    solve_alt(prob)  # was: prob.solve(eps=1e-9, solver=cvx.MOSEK)
    return prob.value


def generate_PPT_operator(d, sep=False, param=100):
    out = np.zeros((d**2, d**2), dtype=complex)

    if sep:
        n = np.random.randint(1, param)
        for i in range(100):
            out += tensor(random_PSD_operator(d), random_PSD_operator(d))
        return out
    else:
        num_tries = param
        for i in range(num_tries):
            U = RandomUnitary(d**2)
            p = np.diag(np.random.rand(d**2))
            P = U @ p @ dag(U)
            if min(eigenvalues(partial_transpose(P, [2], [d, d]))) >= 0:
                out = P
                break
        return out, i


def generate_positive_map(d, num_tries=500, stop=True, display=False, PPT=True):
    if stop:
        out = np.zeros((d**2, d**2), dtype=complex)
        for i in range(num_tries):
            H = random_hermitian_operator(d**2)  # from qutipy
            if bilinear_opt_relax(H, d, d, PPT=PPT) >= 0:
                out = H
                break
        return out, i
    else:
        samples = {}
        hits = []
        for i in range(1, num_tries+1):
            H = random_hermitian_operator(d**2)
            samples[i] = (H, bilinear_opt_relax(H, d, d))
            if samples[i][1] >= 0:
                hits.append(i)
            if display:
                print(i, samples[i][1])
        return samples, hits


def random_hermitian_operator_alt(d):
    U = RandomUnitary(d)
    p = np.random.randn(d)
    return U @ np.diag(p) @ dag(U)


def decomposable(N, dA, dB, opt=False, return_var=False):
    if opt:
        T1 = cvx.Variable((dA*dB, dA*dB), hermitian=True)
        T2 = cvx.Variable((dA*dB, dA*dB), hermitian=True)
        S = cvx.Variable((dA*dB, dA*dB), hermitian=True)
        Y = cvx.Variable((dA*dB, dA*dB), hermitian=True)

        c = [T1 >> 0, T2 >> 0, N - T1 - cvx.partial_transpose(T2, [dA, dB], 1) == S]
        c += [-Y << S, S << Y, Y >> 0]

        obj = cvx.Minimize(cvx.real(cvx.trace(Y)))
        prob = cvx.Problem(obj, constraints=c)

        solve_alt(prob)  # was: prob.solve(eps=1e-9, solver=cvx.MOSEK)

        if return_var:
            return prob.value, T1.value, T2.value, S.value
        else:
            return prob.value
    else:
        T1 = cvx.Variable((dA*dB, dA*dB), hermitian=True)
        T2 = cvx.Variable((dA*dB, dA*dB), hermitian=True)

        c = [T1 >> 0, T2 >> 0, N == T1 + cvx.partial_transpose(T2, [dA, dB], 1)]

        obj = cvx.Minimize(0)
        prob = cvx.Problem(obj, constraints=c)

        solve_alt(prob)  # was: prob.solve(eps=1e-9)

        if return_var:
            return prob.value, prob.status, T1.value, T2.value
        else:
            return prob.value


def bhattacharya_map(a):
    '''
    family of maps from ref.[5]
    '''
    return np.array([
        [a**2/(1+a**2), 0, 0, 0, -a/(1+a**2), 0, 0, 0, -a**2/(1+a**2)],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1/(1+a**2), 0, 0, 0, 0, 0, 0],
        [0, 0, 0, a**2/(1+a**2), 0, 0, 0, 0, 0],
        [-a/(1+a**2), 0, 0, 0, 1/(1+a**2), 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, -a/(1+a**2), 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, -a/(1+a**2), 0, 1/(1+a**2), 0],
        [-a**2/(1+a**2), 0, 0, 0, 0, 0, 0, 0, a**2/(1+a**2)]
    ])


def bhattacharya_map_mod(a):
    '''
    Equation (11) in our paper
    '''
    return np.array([
        [a**2/(1+a**2), 0, 0, 0, a/(1+a**2), 0, 0, 0, a**2/(1+a**2)],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1/(1+a**2), 0, 0, 0, 0, 0, 0],
        [0, 0, 0, a**2/(1+a**2), 0, 0, 0, 0, 0],
        [a/(1+a**2), 0, 0, 0, 1/(1+a**2), 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, a/(1+a**2), 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, a/(1+a**2), 0, 1/(1+a**2), 0],
        [a**2/(1+a**2), 0, 0, 0, 0, 0, 0, 0, a**2/(1+a**2)]
    ])

def bhattacharya_map_mod_fourdim():
    '''
    The map from Eq.(12) of our paper
    We used this to test PPT² via the command
    PPT_square_test(bhattacharya_map_mod_fourdim(),4,300)
    '''
    return (np.array([
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]
    ])/3)

def breuer_hall_map(m, U):
    '''
    The input dimension is n=2*m, and U is a
    skew-symmetric unitary in dimension n.
    '''
    n = 2*m
    return tensor(eye(n), eye(n)) - gamma(n) - tensor(eye(n), U) @ SWAP([1, 2], [n, n]) @ tensor(eye(n), dag(U))


def robertson_map():
    Y = np.array([[0, -1j], [1j, 0]])
    return breuer_hall_map(2, tensor(eye(2), Y))


def choi_map_M3():
    return np.array([
        [1, 0, 0, 0, -1, 0, 0, 0, -1],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [-1, 0, 0, 0, 1, 0, 0, 0, -1],
        [0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [-1, 0, 0, 0, -1, 0, 0, 0, 1]
    ])


def ha_map_M4(a, b, c, d):
    '''
    This map comes from the paper:
        "A class of atomic positive linear maps in matrix algebras"
        by Kil-Chan Ha

    It is not CCP (conditionally completely positive) for all parameter values,
    '''
    C_phi = np.zeros((16, 16), dtype=float)

    C_phi[0, 0] = a
    C_phi[3, 3] = b

    C_phi[4, 4] = c
    C_phi[5, 5] = a

    C_phi[9, 9] = d
    C_phi[10, 10] = a

    C_phi[12, 12] = a
    C_phi[13, 13] = b

    C_phi[0, 5] = -1
    C_phi[0, 10] = -1
    C_phi[0, 15] = -1

    C_phi[4, 1] = -1
    C_phi[4, 10] = -1
    C_phi[4, 15] = -1

    C_phi[8, 1] = -1
    C_phi[8, 5] = -1
    C_phi[8, 15] = -1

    C_phi[12, 1] = -1
    C_phi[12, 5] = -1
    C_phi[12, 10] = -1

    return C_phi


def check_PPT_squared(P, T, d):
    '''
    P is a positive map, T is a completely positive and completely
    co-positive map. Both are specified in terms of their Choi
    matrices. All maps have the same input and output dimension of d.
    '''
    N = partial_trace(
        tensor(partial_transpose(T, [2], [d, d]), eye(d)) @ tensor(eye(d), P),
        [2], [d, d, d]
    )
    return decomposable(N, d, d, opt=True)


def check_PPT_squared_opt(P, d, return_var=False):
    T1 = cvx.Variable((d*d, d*d), hermitian=True)
    T2 = cvx.Variable((d*d, d*d), hermitian=True)
    S = cvx.Variable((d*d, d*d), hermitian=True)
    Y = cvx.Variable((d*d, d*d), hermitian=True)

    T = cvx.Variable((d**2, d**2), hermitian=True)
    c = [T >> 0, cvx.partial_transpose(T, [d, d], 1) >> 0]

    N = cvx.partial_trace(cvx.kron(cvx.partial_transpose(T, [d, d], 1), eye(d)) @ tensor(eye(d), P), [d, d, d], 1)

    c += [T1 >> 0, T2 >> 0, N - T1 - cvx.partial_transpose(T2, [d, d], 1) == S]
    c += [-Y << S, S << Y, Y >> 0]

    obj = cvx.Minimize(cvx.real(cvx.trace(Y)))
    prob = cvx.Problem(obj, constraints=c)

    solve_alt(prob)  # was: prob.solve(eps=1e-9, solver=cvx.MOSEK)

    if return_var:
        return prob.value, T1.value, T2.value, S.value
    else:
        return prob.value


def exp_choi(N, t, d):
    '''
    N is the Choi representation of a linear map. Input and output
    have the same dimension d.
    '''
    K = choi_to_natural(N, d, d)
    exp_K = expm(t * K)
    exp_N = choi_to_natural(exp_K, d, d)
    return exp_N


def one_param_family(N, K, t, d):
    '''
    N is the Choi representation of a map Phi in dimension d. K is
    an arbitrary d x d linear operator. The corresponding map is
        exp(t*(K(.) + (.)K^† + Phi))
    '''
    M = tensor(eye(d), K) + tensor(np.conj(K), eye(d)) + choi_to_natural(N, d, d)
    exp_M = expm(t * M)
    C = choi_to_natural(exp_M, d, d)
    return C

def one_param_family_noK(N, t, d):
    '''
    same as one_param_family but K=0
    N is the Choi representation of a map Phi in dimension d.
    The corresponding map is exp(t*Phi)
    '''
    M = choi_to_natural(N, d, d)
    exp_M = expm(t * M)
    C = choi_to_natural(exp_M, d, d)
    return C


def one_param_family_test(N, tmin, tmax, steps, d, num_tries=100, display=False):
    '''
    This function tests decomposability and complete positivity for random K's;
    for all t between the arguments tmin and tmax (discretized by steps)
    the function outputs the decomposability value D (0 iff decomposable)
    and the smallest eigenvalue of T (>=0 iff CP) for all tested t
    '''
    tries = {}
    for i in range(1, num_tries+1):
        tries[i] = {}
        K = np.random.randn(d, d) + 1j * np.random.randn(d, d)
        for t in np.linspace(tmin, tmax, steps+1):
            T = one_param_family(N, K, t, d)
            D = decomposable(T, d, d, opt=True)
            tries[i][t] = (K, D)
            if display:
                print(i, t, D)
    return tries


def one_param_family_test_noK(N, tmin, tmax, steps, d,display=False):
    '''
    This function tests decomposability and complete positivity;
    for all t between the arguments tmin and tmax (discretized by steps)
    the function outputs the decomposability value D (0 iff decomposable)
    and the smallest eigenvalue of T (>=0 iff CP) for all tested t
    '''
    tries = {}
    for t in np.linspace(tmin, tmax, steps+1):
        T = one_param_family_noK(N, t, d)
        D = decomposable(T, d, d, opt=True)
        tries[t] = (D)
        if display:
          print(t, D, np.min(np.linalg.eigvalsh(0.5*(T + T.conj().T))))
    return tries


def PPT_square_test(
    N, d, num_tries=10, display=True, *, t0=1.0, max_halvings=10, tol=1e-4,
    return_results=False
):
    '''
    For each random K, search t in {t0, t0/2, ...} for which
    T = one_param_family(N, K, t, d) is non-decomposable (D > tol),
    then compute check_PPT_squared_opt(T, d).
    '''
    results = {}

    for i in range(num_tries + 1):
        K = np.random.randn(d, d) + 1j * np.random.randn(d, d)
        t = float(t0)
        found = False
        D = None
        for _ in range(max_halvings + 1):
            T = one_param_family(N, K, t, d)
            D = decomposable(T, d, d, opt=True)
            if D > tol:
                if display:
                    print(f"[try {i}] t={t:.6g}  decomposable-value={D:.3e}")
                found = True
                break
            t *= 0.5

        if not found:
            continue

        val = check_PPT_squared_opt(T, d, return_var=False)
        results[i] = {"K": K, "t": t, "value": val, "decomp_value": D}

        if display:
            print(f"[try {i}] PPT^2 value={val:.6g}")

    if return_results:
        return results


# Choose one of the following B operators to run an analysis on

---


(1) original bhattacharya_map for alpha=1

Findings: running one_param_family_test_noK(B,0,2,200,dim,display=True) returns something

* decomposable for **all times** (same is true for all random K that were checked so this map is conditionally decomposable)
* not completely positive until t=1.27

In [8]:
dim=3
B=bhattacharya_map(1)
one_param_family_test_noK(B,0,2,200,dim,display=True)

0.0 -1.9971274511963783e-07 -4.127093980575254e-16
0.01 1.0551872720097584e-15 -0.0030741737976744633
0.02 -4.1737335972558917e-16 -0.006116189094505992
0.03 1.1548866430848335e-16 -0.009125793672900652
0.04 -2.2307456976929246e-16 -0.012102731416733455
0.05 -4.913972817379726e-14 -0.015046742278856457
0.06 -7.510926446429503e-12 -0.017957562248301417
0.07 -6.765975885183454e-18 -0.020834923317173445
0.08 1.80433851996238e-11 -0.023678553447232725
0.09 -2.4395203348088655e-15 -0.026488176536160892
0.1 7.759307704680712e-15 -0.029263512383508748
0.11 -1.6481370461608726e-12 -0.032004276656321856
0.12 7.212324782300249e-14 -0.034710180854440706
0.13 -1.4684310283476004e-10 -0.037380932275472074
0.14 -1.7961760813370247e-14 -0.04001623397942784
0.15 3.3766108568104742e-12 -0.042615784753027935
0.16 -8.038382272087329e-13 -0.04517927907366364
0.17 -2.0352923569805747e-14 -0.04770640707301806
0.18 6.283966671280188e-11 -0.05019685450033942
0.19 3.766376858895837e-12 -0.05265030268536393
0.2

{np.float64(0.0): -1.9971274511963783e-07,
 np.float64(0.01): 1.0551872720097584e-15,
 np.float64(0.02): -4.1737335972558917e-16,
 np.float64(0.03): 1.1548866430848335e-16,
 np.float64(0.04): -2.2307456976929246e-16,
 np.float64(0.05): -4.913972817379726e-14,
 np.float64(0.06): -7.510926446429503e-12,
 np.float64(0.07): -6.765975885183454e-18,
 np.float64(0.08): 1.80433851996238e-11,
 np.float64(0.09): -2.4395203348088655e-15,
 np.float64(0.1): 7.759307704680712e-15,
 np.float64(0.11): -1.6481370461608726e-12,
 np.float64(0.12): 7.212324782300249e-14,
 np.float64(0.13): -1.4684310283476004e-10,
 np.float64(0.14): -1.7961760813370247e-14,
 np.float64(0.15): 3.3766108568104742e-12,
 np.float64(0.16): -8.038382272087329e-13,
 np.float64(0.17): -2.0352923569805747e-14,
 np.float64(0.18): 6.283966671280188e-11,
 np.float64(0.19): 3.766376858895837e-12,
 np.float64(0.2): -2.316231836305594e-13,
 np.float64(0.21): 4.8296179318483184e-14,
 np.float64(0.22): -4.705476787493353e-10,
 np.float64(


---

(2) a version of the 3D-bhattacharya_map where the off-diagonals do not have a minus sign (again, alpha=1). Findings: running one_param_family_test_noK(B,0,2,200,dim,display=True) returns something

*  **indecomposable** until t=0.44 (for random K's this threshold fluctuated around 0.25 - 0.35)
*  **not completely positive** until t=1.96

In [10]:
dim=3
B=bhattacharya_map_mod(1)
one_param_family_test_noK(B,0,2,200,dim,display=True)

0.0 -1.9971274511963783e-07 -4.127093980575254e-16
0.01 0.002039486986848626 -0.0030741737976744633
0.02 0.00399997630019889 -0.006116189094505992
0.03 0.005877622637724426 -0.009125793672900652
0.04 0.0076725169483107016 -0.012102731416733455
0.05 0.009384142937793194 -0.015046742278856457
0.06 0.011009620266386995 -0.017957562248301417
0.07 0.012547480710028333 -0.020834923317173445
0.08 0.013996082743428357 -0.023678553447232725
0.09 0.015356523363021199 -0.026488176536160892
0.1 0.016623826834970518 -0.029263512383508748
0.11 0.017798143871001745 -0.032004276656321856
0.12 0.018877933817887348 -0.03471018085444072
0.13 0.019860758028824387 -0.037380932275472074
0.14 0.020750212920309884 -0.04001623397942784
0.15 0.021546752990653586 -0.042615784753027935
0.16 0.022248471343727802 -0.04517927907366364
0.17 0.022855461249819964 -0.04770640707301806
0.18 0.02336592996923127 -0.05019685450033942
0.19 0.02377899482455097 -0.05265030268536393
0.2 0.024093618131318096 -0.05506642850088467

{np.float64(0.0): -1.9971274511963783e-07,
 np.float64(0.01): 0.002039486986848626,
 np.float64(0.02): 0.00399997630019889,
 np.float64(0.03): 0.005877622637724426,
 np.float64(0.04): 0.0076725169483107016,
 np.float64(0.05): 0.009384142937793194,
 np.float64(0.06): 0.011009620266386995,
 np.float64(0.07): 0.012547480710028333,
 np.float64(0.08): 0.013996082743428357,
 np.float64(0.09): 0.015356523363021199,
 np.float64(0.1): 0.016623826834970518,
 np.float64(0.11): 0.017798143871001745,
 np.float64(0.12): 0.018877933817887348,
 np.float64(0.13): 0.019860758028824387,
 np.float64(0.14): 0.020750212920309884,
 np.float64(0.15): 0.021546752990653586,
 np.float64(0.16): 0.022248471343727802,
 np.float64(0.17): 0.022855461249819964,
 np.float64(0.18): 0.02336592996923127,
 np.float64(0.19): 0.02377899482455097,
 np.float64(0.2): 0.024093618131318096,
 np.float64(0.21): 0.024309612040531076,
 np.float64(0.22): 0.02442188237099986,
 np.float64(0.23): 0.02443233410721749,
 np.float64(0.24): 0


---

(3) a 4D-generalization of the map from (2). The idea is to omit only one element on the diagonal, cyclicly. I confirmed its (almost-surely) positivity via testing it of millions of random vectors in mathematica. crucially, just having the sum of two diagonal elements in each diagonal spot yields something which is NOT positive, i.e., one really needs the sum of three diagonal elements everywhere

Findings: running one_param_family_test_noK(B,0,1,100,dim,display=True) returns something

*  **indecomposable** until t=0.24. curiously, this value didn't change much when looking at the test function *with* random K's
*  **not completely positive** until t=1.38

In [11]:
dim=4
B=bhattacharya_map_mod_fourdim()
one_param_family_test_noK(B,0,1,100,dim,display=True)

0.0 -5.3777796875863376e-08 -2.2204460492503136e-16
0.01 0.0011296288360734324 -0.0020489660990511485
0.02 0.0021521644573781907 -0.004075492634001742
0.03 0.0030611607354724246 -0.006079359975486117
0.04 0.003855180590863045 -0.00806034597656516
0.05 0.004555041515873761 -0.010018225948147424
0.06 0.005152024717567646 -0.011952772634152937
0.07 0.0056553053562702306 -0.013863756186417043
0.08 0.006065207526880799 -0.01575094413933169
0.09 0.006380733012788619 -0.017614101384221495
0.1 0.006602109762833017 -0.019452990143451813
0.11 0.006727648856943732 -0.021267369944266312
0.12 0.006760802834196426 -0.023056997592350973
0.13 0.006696597863512994 -0.024821627145122142
0.14 0.006536876347560649 -0.026561009884735618
0.15 0.006281092597276721 -0.028274894290813722
0.16 0.005927221117903749 -0.029963026012888166
0.17 0.00547271878684611 -0.031625147842555074
0.18 0.004919756095674995 -0.0332609996853398
0.19 0.004269871416947229 -0.034870318532268324
0.2 0.0035178574458659263 -0.03645283

{np.float64(0.0): -5.3777796875863376e-08,
 np.float64(0.01): 0.0011296288360734324,
 np.float64(0.02): 0.0021521644573781907,
 np.float64(0.03): 0.0030611607354724246,
 np.float64(0.04): 0.003855180590863045,
 np.float64(0.05): 0.004555041515873761,
 np.float64(0.06): 0.005152024717567646,
 np.float64(0.07): 0.0056553053562702306,
 np.float64(0.08): 0.006065207526880799,
 np.float64(0.09): 0.006380733012788619,
 np.float64(0.1): 0.006602109762833017,
 np.float64(0.11): 0.006727648856943732,
 np.float64(0.12): 0.006760802834196426,
 np.float64(0.13): 0.006696597863512994,
 np.float64(0.14): 0.006536876347560649,
 np.float64(0.15): 0.006281092597276721,
 np.float64(0.16): 0.005927221117903749,
 np.float64(0.17): 0.00547271878684611,
 np.float64(0.18): 0.004919756095674995,
 np.float64(0.19): 0.004269871416947229,
 np.float64(0.2): 0.0035178574458659263,
 np.float64(0.21): 0.0026644763245098346,
 np.float64(0.22): 0.001709147241072159,
 np.float64(0.23): 0.0006569170043878666,
 np.float6


---

(4) random CP map as a sanity check. as expected, the one-parameter family is CP and decomposable for all times

In [13]:
dim=4
B=random_PSD_operator(dim*dim)
one_param_family_test_noK(B,0,0.1,10,dim,display=True)

0.0 -5.3777796875863376e-08 -2.2204460492503136e-16
0.01 1.887984670867972e-09 0.27599197771921613
0.02 3.850125421671635e-09 1.640506550980373
0.03 1.0192143349174137e-09 6.897376074758282
0.04 -2.3472594074647294e-10 26.702423987327812
0.05 -1.875978703528131e-12 103.85162877918951
0.06 -5.870739098585874e-12 409.16985927859395
0.07 -8.492306977171175e-11 1620.4925558900538
0.08 -7.063172140935369e-11 6428.45658492646
0.09 -5.395368474353912e-10 25514.1368087424
0.1 -2.1116847260877748e-10 101278.635806191


{np.float64(0.0): -5.3777796875863376e-08,
 np.float64(0.01): 1.887984670867972e-09,
 np.float64(0.02): 3.850125421671635e-09,
 np.float64(0.03): 1.0192143349174137e-09,
 np.float64(0.04): -2.3472594074647294e-10,
 np.float64(0.05): -1.875978703528131e-12,
 np.float64(0.06): -5.870739098585874e-12,
 np.float64(0.07): -8.492306977171175e-11,
 np.float64(0.08): -7.063172140935369e-11,
 np.float64(0.09): -5.395368474353912e-10,
 np.float64(0.1): -2.1116847260877748e-10}



---



# Detailed analysis start



---



In [22]:
dim=4
B=bhattacharya_map_mod_fourdim()

CP check (CP iff >=0)

In [23]:
np.min(eigenvalues(B))

np.float64(-0.20601132958329837)

---

ccp check (>=0 iff ccp)

In [24]:
np.min(eigenvalues((eye(dim*dim)-max_ent(dim))@B@(eye(dim*dim)-max_ent(dim))))

np.float64(-0.20601132958329826)

---

decomposability check

(decomposable iff 0 / not decomposable iff inf)

In [25]:
decomposable(B,dim,dim,opt=False,return_var=False)

inf

(decomposable iff 0)

In [26]:
decomposable(B,dim,dim,opt=True)

0.16962315140580642

---

PPT² check (counterexample iff non-zero)

In [27]:
check_PPT_squared_opt(B,dim,return_var=False)

-5.624144088565098e-13

PPT² check with random K's

In [32]:
PPT_square_test(B,dim,300)

[try 0] t=0.125  decomposable-value=4.058e-03
[try 0] PPT^2 value=-6.18606e-13
[try 1] t=0.125  decomposable-value=2.332e-03
[try 1] PPT^2 value=-5.44676e-13
[try 2] t=0.125  decomposable-value=2.855e-03
[try 2] PPT^2 value=-5.62414e-13
[try 3] t=0.125  decomposable-value=3.191e-03
[try 3] PPT^2 value=-6.38718e-13
[try 4] t=0.125  decomposable-value=3.407e-03
[try 4] PPT^2 value=-5.81058e-13
[try 5] t=0.125  decomposable-value=3.001e-03
[try 5] PPT^2 value=-3.82669e-13
[try 6] t=0.125  decomposable-value=1.930e-03
[try 6] PPT^2 value=-4.79865e-13
[try 7] t=0.125  decomposable-value=4.099e-03
[try 7] PPT^2 value=-5.48712e-13
[try 8] t=0.125  decomposable-value=3.978e-03
[try 8] PPT^2 value=-5.98584e-13
[try 9] t=0.125  decomposable-value=3.624e-03
[try 9] PPT^2 value=-6.7413e-13
[try 10] t=0.125  decomposable-value=2.674e-03
[try 10] PPT^2 value=-5.31196e-13
[try 11] t=0.125  decomposable-value=4.395e-03
[try 11] PPT^2 value=-5.49146e-13
[try 12] t=0.125  decomposable-value=4.784e-03
[t

---

decomposability/CP check for one-parameter semigroup **for random K's**. tests the map B for t between the following two arguments, with a certain step size

output is: (current attempt) (current t) (decomposability: decomposable iff 0)

In [31]:
one_param_family_test(B,0,1,20,dim,num_tries=3,display=True);

1 0.0 -5.3777796875863376e-08
1 0.05 0.004432074062059954
1 0.1 0.00491835175150884
1 0.15000000000000002 0.00034377870655439164
1 0.2 1.8137854022658931e-12
1 0.25 -9.41131254517162e-11
1 0.30000000000000004 -5.456686911922808e-10
1 0.35000000000000003 -1.8126544883588265e-10
1 0.4 1.169093238828273e-08
1 0.45 1.807561264790806e-09
1 0.5 -5.49430569304049e-09
1 0.55 -1.6347622615777364e-09
1 0.6000000000000001 1.0967576531333686e-10
1 0.65 -1.8618558170728102e-10
1 0.7000000000000001 4.685234743413964e-10
1 0.75 -3.4014845901804536e-11
1 0.8 -1.0953428246723315e-12
1 0.8500000000000001 -2.5936086127364693e-09
1 0.9 2.472673879281547e-09
1 0.9500000000000001 -3.70413819999888e-13
1 1.0 -1.544841747919619e-12
2 0.0 -5.3777796875863376e-08
2 0.05 0.004492850446899055
2 0.1 0.00551006224963043
2 0.15000000000000002 0.002302858408066547
2 0.2 4.329852068395184e-14
2 0.25 -5.380379567074613e-09
2 0.30000000000000004 -1.5211414628261327e-12
2 0.35000000000000003 2.25957664117217e-09
2 0.4 -2

---

#exp-exp

Use the non-decomposable output of the semigroup method as the **input** again. The following was done with map (3) and t=0.1 (because the decomposability region was until ~0.22, and the decomposability value was highest around 0.1)

first it generated this map (Bexp), then it checks whether Bexp is really decomposable (it should be by the previous test), and then it re-runs the one-parameter family tests

In [ ]:
K = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
Bexp = one_param_family(B, 0*K, 0.1, dim)
Bexp2= one_param_family(Bexp, 0*K, 0.6, dim)
Bexp3= one_param_family(Bexp2, 0*K, 0.2, dim)
Bexp4= one_param_family(Bexp3, 0*K, 0.2, dim)

In [ ]:
np.min(eigenvalues((eye(dim*dim)-max_ent(dim))@Bexp@(eye(dim*dim)-max_ent(dim))))

np.complex128(-0.023056997592350956+0j)

In [ ]:
decomposable(Bexp,dim,dim,opt=True)

0.0030897550344318697

In [ ]:
check_PPT_squared_opt(Bexp4,dim,return_var=False)

-6.433021473156505e-13

In [ ]:
one_param_family_test_noK(Bexp3,0,0.5,5,dim,display=True);

In [ ]:
one_param_family_test(Bexp,0,0.5,10,dim,num_tries=10,display=True);

1 0.0 -1.9971274511963783e-07
1 0.05 0.0010357407493822848
1 0.1 0.0016741799286403816
1 0.15000000000000002 0.0017392581062019867
1 0.2 0.0011746648000191995
1 0.25 0.000109796106654209
1 0.30000000000000004 -3.001130770723795e-14
1 0.35000000000000003 -1.95935666549235e-13
1 0.4 -5.501511953414521e-11
1 0.45 -2.768946322576242e-11
1 0.5 1.0594251342242305e-11
2 0.0 -1.9971274511963783e-07
2 0.05 0.0011294581533834523
2 0.1 0.0020471724412793996
2 0.15000000000000002 0.002469669655264175
2 0.2 0.002155239449220251
2 0.25 0.0010606148062510276
2 0.30000000000000004 -3.5256606207389056e-15
2 0.35000000000000003 1.4567163220862305e-14
2 0.4 1.755473663142043e-14
2 0.45 -2.9460579057783034e-12
2 0.5 6.3242762583045085e-12
3 0.0 -1.9971274511963783e-07
3 0.05 0.0009618717052412113
3 0.1 0.0014967259252493105
3 0.15000000000000002 0.0015735924635714013
3 0.2 0.001212898576163036
3 0.25 0.0005422822736902298
3 0.30000000000000004 -9.196468639122575e-15
3 0.35000000000000003 2.012179863929356